In [ ]:
# !pip install rdkit-pypi

In [6]:
import pandas as pd
import numpy as np
import csv
import os, sys
import json
import joblib
import argparse
from rdkit import Chem
import openbabel
import pybel
from rdkit.Chem import rdMolDescriptors

In [8]:
import oddt.toolkits.extras.rdkit as ordkit

In [9]:
df_1 = pd.read_csv(r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\All_csv_files\Final_set.csv', index_col=False)
df_1 = df_1.drop(columns='Unnamed: 0')
display(df_1)

,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,canonical_smiles_x,document_chembl_id,molecule_chembl_id_x,...,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,IdList,value_in_ug_ml
0,748926,CHEMBL810377,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,CHEMBL1436,...,0,0,0,0,0,0,0,0,1280.0,1.00
1,748926,CHEMBL810377,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,CHEMBL1436,...,0,0,0,0,0,0,0,0,1280.0,1.00
2,748927,CHEMBL810376,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,CHEMBL1436,...,0,0,0,0,0,0,0,0,1280.0,2.00
3,748927,CHEMBL810376,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,CHEMBL1436,...,0,0,0,0,0,0,0,0,1280.0,2.00
4,748928,CHEMBL883635,In vitro antibacterial activity against the gr...,F,BAO_0002146,BAO_0000218,organism-based format,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,CHEMBL1132087,CHEMBL1436,...,0,0,0,0,0,0,0,0,1280.0,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,25103833,CHEMBL5262798,Antimycobacterial activity against Mycobacteri...,F,BAO_0002146,BAO_0000218,organism-based format,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...,CHEMBL5260811,CHEMBL148,...,0,0,0,0,0,0,0,0,1773.0,0.16
233230,25103833,CHEMBL5262798,Antimycobacterial activity against Mycobacteri...,F,BAO_0002146,BAO_0000218,organism-based format,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...,CHEMBL5260811,CHEMBL148,...,0,0,0,0,0,0,0,0,1773.0,0.16
233231,25103833,CHEMBL5262798,Antimycobacterial activity against Mycobacteri...,F,BAO_0002146,BAO_0000218,organism-based format,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...,CHEMBL5260811,CHEMBL148,...,0,0,0,0,0,0,0,0,1773.0,0.16
233232,25103833,CHEMBL5262798,Antimycobacterial activity against Mycobacteri...,F,BAO_0002146,BAO_0000218,organism-based format,C[C@@H](O)[C@H]1C(=O)N2C(C(=O)O)=C(SCCNC=N)C[C...,CHEMBL5260811,CHEMBL148,...,0,0,0,0,0,0,0,0,1773.0,0.16


In [10]:
molname_list = df_1[['molecule_chembl_id_x','canonical_smiles_x']]
molname_list = molname_list.drop_duplicates()
# Сброс индексов
molname_list = molname_list.reset_index(drop=True)
display(molname_list)

,molecule_chembl_id_x,canonical_smiles_x
0,CHEMBL1436,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
1,CHEMBL1435,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)Cn4cn...
2,CHEMBL1434,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...
3,CHEMBL1433,C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C...
4,CHEMBL1630575,CC[C@H](C)C[C@H](C)CCCCCCCCC(=O)N[C@H]1C[C@@H]...
...,...,...
69,CHEMBL106,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F
70,CHEMBL430,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...
71,CHEMBL529,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...
72,CHEMBL158,C[C@H]1[C@H](NC(=O)/C(=N\OC(C)(C)C(=O)O)c2csc(...


In [11]:
smiles = df_1[["canonical_smiles_x"]]
smiles = smiles.drop_duplicates()
smiles = smiles.reset_index(drop=True)
display(smiles)

,canonical_smiles_x
0,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...
1,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)Cn4cn...
2,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...
3,C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C...
4,CC[C@H](C)C[C@H](C)CCCCCCCCC(=O)N[C@H]1C[C@@H]...
...,...
69,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F
70,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...
71,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...
72,C[C@H]1[C@H](NC(=O)/C(=N\OC(C)(C)C(=O)O)c2csc(...


In [12]:
import pandas as pd
import requests

# Функция для получения IUPAC имени через PubChem API
def get_iupac_name(smiles):
    try:
        # Запрос к PubChem для получения информации по SMILES
        url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{smiles}/property/IUPACName/JSON"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            return data['PropertyTable']['Properties'][0]['IUPACName']
        else:
            return "IUPAC не найдено"
    except Exception as e:
        print(f"Ошибка для SMILES {smiles}: {e}")
        return "Ошибка"

# Применяем функцию к столбцу
smiles['molecule'] = smiles['canonical_smiles_x'].apply(get_iupac_name)

# Вывод результата
display(smiles)

,canonical_smiles_x,molecule
0,CO/N=C(\C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(COC(N)...,IUPAC не найдено
1,Cc1nnc(SCC2=C(C(=O)O)N3C(=O)[C@@H](NC(=O)Cn4cn...,"(6R,7R)-3-[(5-methyl-1,3,4-thiadiazol-2-yl)sul..."
2,CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(O...,"(4S,4aS,5aR,12aR)-4,7-bis(dimethylamino)-1,10,..."
3,C[C@H]1c2cccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(C...,"(4S,4aR,5S,5aR,6R,12aR)-4-(dimethylamino)-1,5,..."
4,CC[C@H](C)C[C@H](C)CCCCCCCCC(=O)N[C@H]1C[C@@H]...,"(10R,12S)-N-[(3S,6S,9S,11R,15S,18S,20R,21S,24S..."
...,...,...
69,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F,"2-(2,4-difluorophenyl)-1,3-bis(1,2,4-triazol-1..."
70,CO/N=C1\CN(c2nc3c(cc2F)c(=O)c(C(=O)O)cn3C2CC2)...,IUPAC не найдено
71,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...,"(2R,3S,4R,5R,8R,10R,11R,12S,13S,14R)-11-[(2S,3..."
72,C[C@H]1[C@H](NC(=O)/C(=N\OC(C)(C)C(=O)O)c2csc(...,IUPAC не найдено


In [14]:
# # # Сохраняем DataFrame в файл с расширением
# smiles.to_csv('smiles.smi', index=False)
# # Сохраняем DataFrame в файл smiles.smi
# file_path = '/content/smiles.txt'
# smiles.to_csv(file_path, index=False)

# file_path

# Сохраняем DataFrame в файл smiles.txt в нужном формате
file_path = r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\Learning\smiles.txt'
smiles.to_csv(file_path, index=False, header=False, sep='\t')

# Выводим путь к файлу
print(f"Data saved to {file_path}")

Data saved to C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\Learning\smiles.txt


In [15]:
# # Функция для извлечения GAFF фичей для каждой молекулы
# def mol_gaff_features_from_smiles(smiles):
#     atom_list = ['c', 'c1', 'c2', 'c3', 'ca', 'cp', 'cq', 'cc', 'cd', 'ce', 'cf', 'cg', 'ch', 'cx', 'cy', 'cu', 'cv', 'cz',
#                 'h1', 'h2', 'h3', 'h4', 'h5', 'ha', 'hc', 'hn', 'ho', 'hp', 'hs', 'hw', 'hx', 'f', 'cl', 'br', 'i', 'n', 'n1',
#                 'n2', 'n3', 'n4', 'na', 'nb', 'nc', 'nd', 'ne', 'nf', 'nh', 'no', 'o', 'oh', 'os', 'ow', 'p2', 'p3', 'p4', 'p5',
#                 'pb', 'pc', 'pd', 'pe', 'pf', 'px', 'py', 's', 's2', 's4', 's6', 'sh', 'ss', 'sx', 'sy']

#     # Список для сохранения всех фичей
#     all_features = []

#     # Проход по каждой строке с SMILES
#     for smiles_string in smiles['canonical_smiles'].astype(str):
#         features = []

#         # Создаем объект молекулы с помощью Pybel
#         molecule = pybel.readstring("smi", smiles_string)

#         # Инициализация силы поля GAFF
#         force_field = openbabel.OBForceField.FindForceField("GAFF")
#         force_field.Setup(molecule.OBMol)
#         force_field.GetAtomTypes(molecule.OBMol)

#         # Генерация признаков для каждого атома
#         for i in range(molecule.OBMol.NumAtoms()):
#             at = molecule.OBMol.GetAtom(i + 1)
#             try:
#                 t = at.GetData("FFAtomType")  # получаем тип атома
#                 atom_type = str(t.GetValue())
#                 # Преобразуем тип атома в one-hot вектор
#                 atom_type_f = one_of_k_encoding_unk(atom_type, atom_list)
#             except:
#                 print("[unknown gaff atom type] " + smiles_string)
#                 atom_type_f = [0] * len(atom_list)  # Если тип неизвестен, создаем нулевой вектор
#             f = np.array(atom_type_f, dtype=np.float32)
#             features.append(f)

#         # Добавляем фичи для каждой молекулы в общий список
#         all_features.append(features)

#     return all_features
def mol_gaff_features(mol):
    import pybel
    atom_list=['c', 'c1', 'c2', 'c3', 'ca', 'cp', 'cq', 'cc', 'cd', 'ce', 'cf', 'cg', 'ch', 'cx', 'cy', 'cu', 'cv', 'cz',
            'h1', 'h2', 'h3', 'h4', 'h5', 'ha', 'hc', 'hn', 'ho', 'hp', 'hs', 'hw', 'hx', 'f', 'cl', 'br', 'i', 'n', 'n1',
            'n2', 'n3', 'n4', 'na', 'nb', 'nc', 'nd', 'ne', 'nf', 'nh', 'no', 'o', 'oh', 'os', 'ow', 'p2', 'p3', 'p4', 'p5',
            'pb', 'pc', 'pd', 'pe', 'pf', 'px', 'py', 's', 's2', 's4', 's6', 'sh', 'ss', 'sx', 'sy']
    smiles = Chem.MolToSmiles(mol)
    molecule = pybel.readstring("smi", smiles)
    force_field = pybel._forcefields["gaff"]
    force_field.Setup(molecule.OBMol)
    force_field.GetAtomTypes(molecule.OBMol)
    features=[]
    for i in range(molecule.OBMol.NumAtoms()):
        at=molecule.OBMol.GetAtom(i+1)
        try:
            t = at.GetData("FFAtomType") # an OBPairData object
            atom_type=str(t.GetValue())
            atom_type_f = one_of_k_encoding_unk(atom_type, atom_list)
        except:
            print("[unknown gaff atom type] "+smiles)
            atom_type_f = [0]*len(atom_list)
        f = np.array(atom_type_f,dtype=np.float32)
        features.append(f)
    return features

In [16]:
def atom_features(atom, bool_id_feat=False, use_sybyl=True, explicit_H=False, generative_mode=True):
    if bool_id_feat:
        return np.array([atom_to_id(atom)])
    else:
        if use_sybyl:
            import oddt.toolkits.extras.rdkit as ordkit
            atom_type = ordkit._sybyl_atom_type(atom)
            atom_list = ['C.ar', 'C.cat', 'C.1', 'C.2', 'C.3', 'N.ar', 'N.am', 'N.pl3', 'N.1', 'N.2', 'N.3', 'N.4', 'O.co2',
                         'O.2', 'O.3', 'S.o', 'S.o2', 'S.2', 'S.3', 'F', 'Si', 'P', 'P3', 'Cl', 'Br', 'Mg', 'Na', 'Ca',
                         'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn',
                         'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr', 'Pt', 'Hg', 'Pb', 'Unknown']
        else:
            atom_type = atom.GetSymbol()
            atom_list=['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V',
                       'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni',
                       'Cd', 'In', 'Mn', 'Zr', 'Cr', 'Pt', 'Hg', 'Pb', 'Unknown']
        results = one_of_k_encoding_unk(atom_type, atom_list) + \
            one_of_k_encoding_unk(atom.GetDegree(),[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]) + \
            one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5, 6]) + \
            [atom.GetFormalCharge(), atom.GetNumRadicalElectrons()] + \
            one_of_k_encoding_unk(atom.GetHybridization(),
                                  [Chem.rdchem.HybridizationType.SP, Chem.rdchem.HybridizationType.SP2,
                                   Chem.rdchem.HybridizationType.SP3, Chem.rdchem.HybridizationType.
                                   SP3D, Chem.rdchem.HybridizationType.SP3D2 ]) + \
            [atom.GetIsAromatic()]

    if generative_mode:
        results+=[atom.IsInRing()]+[atom.IsInRingSize(i) for i in range(3,8)]
    # In case of explicit hydrogen(QM8, QM9), avoid calling `GetTotalNumHs`
    if not explicit_H:
        results+= one_of_k_encoding_unk(atom.GetTotalNumHs(),[0, 1, 2, 3, 4])

    return (results)
    # return np.array(results) that it was before

In [17]:
# Ваша функция для one-hot кодирования
def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    allowable_set = ['C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg', 'Na', 'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V',
                       'K', 'Tl', 'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H', 'Li', 'Ge', 'Cu', 'Au', 'Ni',
                       'Cd', 'In', 'Mn', 'Zr', 'Cr', 'Pt', 'Hg', 'Pb', 'Unknown']
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

In [18]:
def dense_to_sparse(dense):
    from scipy.sparse import coo_matrix
    coo = coo_matrix(dense)
    sh = coo.shape
    val = coo.data
    sp = list(zip(coo.row, coo.col))
    return np.array(sp), np.array(val, dtype=np.float32), np.array(sh)


In [19]:
def create_multi_adjancy_matrix(mol):
    #mol_adj = Chem.GetAdjacencyMatrix(mol,useBO=True)
    num=mol.GetNumAtoms()
    nch=6
    adj = np.zeros((nch,num,num), dtype=np.int64)
    for b in mol.GetBonds():
        i=b.GetBeginAtomIdx()
        j=b.GetEndAtomIdx()
        t=b.GetBondType()
        if b.GetIsConjugated():
            ch=5
            adj[ch,i,j]=1
        if t==Chem.rdchem.BondType.SINGLE:
            ch=0
            adj[ch,i,j]=1
        elif t==Chem.rdchem.BondType.DOUBLE:
            ch=1
            adj[ch,i,j]=1
        elif t==Chem.rdchem.BondType.TRIPLE:
            ch=2
            adj[ch,i,j]=1
        elif t==Chem.rdchem.BondType.AROMATIC:
            ch=3
            adj[ch,i,j]=1
        else:
            ch=4
            adj[ch,i,j]=1
    for ch in range(nch):
        for i in range(num):
            adj[ch][i][i] = int(1)
    return adj


In [20]:
def create_adjancy_matrix(mol):
    mol_adj = Chem.GetAdjacencyMatrix(mol)
    row_num = len(mol_adj)
    adj = np.array(mol_adj, dtype=np.int64)
    for i in range(row_num):  # Set diagonal elements to 1, fill others with the adjacency matrix from RDkit
        adj[i][i] = int(1)
    return adj

In [21]:
def create_feature_matrix(mol, atom_num_limit, use_sybyl=True):
    if use_sybyl:
        Chem.GetSymmSSSR(mol)
    feature = [atom_features(atom) for atom in mol.GetAtoms()]
    for _ in range(atom_num_limit - len(feature)):
        feature.append(np.zeros(len(feature[0]), dtype=np.int64))
    return feature

In [22]:
# def create_gaff_feature_matrix(mol, atom_num_limit):
#     # Преобразуем GAFF features в список
#     feature = list(smiles['GAFF_features'])

#     # Преобразуем все элементы в списки
#     feature = [f.tolist() if isinstance(f, np.ndarray) else f for f in feature]

#     # Добавляем недостающие нулевые элементы, если нужно
#     for i in range(atom_num_limit - len(feature)):
#         feature.append([0] * len(feature[0]))  # Добавляем нулевые списки вместо массивов

#     return feature
def create_gaff_feature_matrix(mol, atom_num_limit):
    #Chem.SanitizeMol(mol)
    feature = mol_gaff_features(mol)
    for _ in range(atom_num_limit - len(feature)):
        feature.append(np.zeros(len(feature[0]), dtype=np.int))
    return feature


In [ ]:
# import argparse
# import joblib
# import numpy as np
# from rdkit import Chem
# from rdkit.Chem import AllChem

# from scipy.sparse import coo_matrix

In [ ]:
# def dense_to_sparse(adj):
#           row, col = np.where(adj)
#           data = adj[row, col]
#           coo = coo_matrix((data, (row, col)), shape=adj.shape)
#           return coo

In [23]:
from rdkit import Chem
import joblib
import numpy as np

def main():
    # Параметры
    input_path = r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\Learning\smiles.txt'  # Путь к входному файлу
    output_path = r'C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\Learning\dataset.jbl'  # Путь к выходному файлу
    atom_num_limit = 70  # Ограничение на количество атомов
    multi = False  # Многомолекулярный режим
    use_gaff = False  # Использовать GAFF

    # Чтение входного файла
    with open(input_path, "r") as f:
        lines = f.readlines()
        mol_obj_list = []
        mol_name_list = []

        for line in lines:
            parts = line.split(" ")
            if len(parts) < 1:
                continue
            try:
                mol = Chem.MolFromSmarts(parts[0])
                if mol is None:
                    print(f"Failed to parse SMARTS: {parts[0]}")
                    continue  # Пропустить некорректные молекулы
                mol_obj_list.append(mol)
                mol_name_list.append(parts[1] if len(parts) > 1 else f"index{len(mol_obj_list):04d}")
            except Exception as e:
                print(f"Error processing line: {line}. Error: {e}")

    atom_num_list = []
    adj_list = []
    feature_list = []

    # Обработка молекул
    for mol in mol_obj_list:
        try:
            Chem.SanitizeMol(mol, sanitizeOps=Chem.SANITIZE_ADJUSTHS)
        except Exception as e:
            print(f"Sanitization failed for molecule. Error: {e}")
            continue  # Пропустить молекулу в случае ошибки

        if mol is None:
            continue
        if atom_num_limit is not None and mol.GetNumAtoms() > atom_num_limit:
            continue
        if not multi:
            adj = create_adjancy_matrix(mol)
            adj_list.append(dense_to_sparse(adj))
        else:
            adj = create_multi_adjancy_matrix(mol)
            adjs = [dense_to_sparse(a) for a in adj]
            adj_list.append(adjs)
        if use_gaff:
            feature = create_gaff_feature_matrix(mol, atom_num_limit)
        else:
            feature = create_feature_matrix(mol, atom_num_limit)
        atom_num_list.append(mol.GetNumAtoms())
        feature_list.append(feature)

    # Преобразование в массив NumPy
    feature_list = np.array(feature_list)
    max_node_num = atom_num_limit
    obj_output = True
    if obj_output:
        obj = {
            "feature": feature_list,
            "adj": adj_list,
            "max_node_num": max_node_num
        }

        print('max_node_num', obj["max_node_num"])
        print('feature', feature_list.shape)
        mol_info = {"obj_list": mol_obj_list, "name_list": mol_name_list}
        obj.update(mol_info)
        print('[SAVE]', output_path)
        joblib.dump(obj, output_path, compress=3)

if __name__ == '__main__':
    main()



max_node_num 70
feature (67, 70, 229)
[SAVE] C:\Users\Igorr\Documents\ITMO5grade\Project_with_Susan\Made_code_github\Made_code\Learning\dataset.jbl


In [2]:
import tensorflow as tf

In [6]:


if tf.__version__.split(".")[0] == "2":
    import tensorflow.compat.v1 as tf

    tf.disable_v2_behavior()
    import tensorflow.keras as K
else:
    import tensorflow.contrib.keras as K
import numpy as np
import joblib
import kgcn.layers
from tensorflow.python.keras.layers import Dense


def build_placeholders(info, config, batch_size=4, **kwargs):
    adj_channel_num = info.adj_channel_num
    encoder_output_dim = 64
    preference_list_length = 2
    adjs = [
        [
            [
                tf.sparse_placeholder(
                    tf.float32, name="adj_" + str(a) + "_" + str(b) + "_" + str(p)
                )
                for a in range(adj_channel_num)
            ]
            for b in range(batch_size)
        ]
        for p in range(preference_list_length)
    ]
    placeholders = {
        "adjs": adjs,
        "epsilon": tf.placeholder(
            tf.float32,
            shape=(batch_size, info.graph_node_num, encoder_output_dim),
            name="epsilon",
        ),
        "mask": [
            tf.placeholder(tf.float32, shape=(batch_size,), name="mask" + "_" + str(p))
            for p in range(preference_list_length)
        ],
        "dropout_rate": tf.placeholder(tf.float32, name="dropout_rate"),
        "enabled_node_nums": tf.placeholder(
            tf.int32, shape=(batch_size,), name="enabled_node_nums"
        ),
        "is_train": tf.placeholder(tf.bool, name="is_train"),
    }
    if info.feature_enabled:
        placeholders["features"] = [
            tf.placeholder(
                tf.float32,
                shape=(batch_size, info.graph_node_num, info.feature_dim),
                name="feature" + "_" + str(p),
            )
            for p in range(preference_list_length)
        ]
    else:
        placeholders["features"] = None
    return placeholders


def encode(name, inputs, info, batch_size):
    internal_dim = 64
    encoder_output_dim = inputs["encoder_output_dim"]
    in_adjs = inputs["adjs"]
    features = inputs["features"]
    dropout_rate = inputs["dropout_rate"]
    is_train = inputs["is_train"]
    enabled_node_nums = inputs["enabled_node_nums"]
    adj_channel_num = info.adj_channel_num

    with tf.variable_scope(name):
        layer = features
        layer = kgcn.layers.GraphConv(internal_dim, adj_channel_num)(layer, adj=in_adjs)
        layer = kgcn.layers.GraphBatchNormalization()(
            layer, max_node_num=info.graph_node_num, enabled_node_nums=enabled_node_nums
        )
        layer = tf.tanh(layer)
        layer = kgcn.layers.GraphConv(internal_dim, adj_channel_num)(layer, adj=in_adjs)
        layer = kgcn.layers.GraphBatchNormalization()(
            layer, max_node_num=info.graph_node_num, enabled_node_nums=enabled_node_nums
        )
        layer = tf.tanh(layer)
        layer = kgcn.layers.GraphDense(internal_dim)(layer)
        layer = tf.sigmoid(layer)
        layer = kgcn.layers.GraphGather()(layer)

        mean_layer = Dense(encoder_output_dim, kernel_initializer="random_uniform")(
            layer
        )
        std_layer = Dense(encoder_output_dim)(layer)
        std_layer = tf.nn.softplus(std_layer)
        std_layer = tf.sqrt(std_layer)
        mean_layer = tf.clip_by_value(mean_layer, -100, 100)
        std_layer = tf.clip_by_value(std_layer, -5, 5)
    return mean_layer, std_layer


def decode_nodes(name, inputs, info):
    dropout_rate = inputs["dropout_rate"]
    layer = inputs["input_layer"]
    input_dim = inputs["input_layer_dim"]
    decoded_output_dim = inputs["output_layer_dim"]
    node_num = inputs["decoded_node_num"]
    is_train = inputs["is_train"]
    enabled_node_nums = inputs["enabled_node_nums"]
    with tf.variable_scope(name):
        layer = kgcn.layers.GraphDense(
            decoded_output_dim, kernel_initializer="random_uniform", name="dense_1"
        )(layer)
    return layer


def decode_links(name, inputs, info):
    dropout_rate = inputs["dropout_rate"]
    internal_dim = 64
    layer = inputs["input_layer"]
    input_dim = inputs["input_layer_dim"]
    is_train = inputs["is_train"]
    node_num = inputs["decoded_node_num"]
    enabled_node_nums = inputs["enabled_node_nums"]
    with tf.variable_scope(name):
        layer = kgcn.layers.GraphDense(internal_dim, name="dense_1")(layer)
        layer = kgcn.layers.GraphBatchNormalization(name="bn_1")(
            layer, max_node_num=info.graph_node_num, enabled_node_nums=enabled_node_nums
        )
        layer = tf.sigmoid(layer)
        layer = kgcn.layers.GraphDense(internal_dim, name="dense_2")(layer)
        layer = tf.sigmoid(layer)
        # layer=kgcn.layers.GraphDecoderInnerProd()(layer)
        layer = kgcn.layers.GraphDecoderDistMult()(layer)
    return layer


def print_variables():
    # print variables
    print("## training variables")
    vars_em = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="nn")
    for v in vars_em:
        print(v.name)


# TODO: hard coding parameters
def build_model(placeholders, info, config, batch_size=4, **kwargs):
    adj_channel_num = info.adj_channel_num
    ## compute output 0
    if not info.feature_enabled:
        print("[ERROR] not supported yet")
        quit()
    # encoder
    features = placeholders["features"][0]
    mask = placeholders["mask"][0]
    encoder_output_dim = 64
    input_encoder = {
        "adjs": placeholders["adjs"][0],
        "features": features,
        "encoder_output_dim": encoder_output_dim,
        "dropout_rate": placeholders["dropout_rate"],
        "is_train": placeholders["is_train"],
        "enabled_node_nums": placeholders["enabled_node_nums"],
    }

    layer_mean, layer_std = encode(
        "encode_nn", input_encoder, info, batch_size=batch_size
    )
    layer_mean1 = layer_mean
    # layer_mean: batch_size x dim
    # generating node_num vectors
    layer_mean = tf.reshape(layer_mean, (batch_size, 1, encoder_output_dim))
    layer_mean = tf.tile(layer_mean, (1, info.graph_node_num, 1))
    layer_std = tf.reshape(layer_std, (batch_size, 1, encoder_output_dim))
    layer_std = tf.tile(layer_std, (1, info.graph_node_num, 1))

    layer = layer_mean + layer_std * placeholders["epsilon"]  # reparameterization trick
    # TODO: use stable cost function
    e = 1.0e-10
    klqp_loss_el = 1 + 2 * tf.log(layer_std + e) - layer_mean**2 - layer_std
    klqp_loss_el = tf.reduce_sum(klqp_loss_el, axis=2)
    klqp_loss_el = tf.reduce_sum(klqp_loss_el, axis=1)
    klqp_loss = -1 / 2.0 * tf.reduce_mean(klqp_loss_el, axis=0)
    # layer: batch_size x node_num x dim
    ## decoder
    decoded_node_num = info.graph_node_num
    input_decoder = {
        "input_layer": layer,
        "input_layer_dim": 64,
        "output_layer_dim": features.shape[2],
        "decoded_node_num": decoded_node_num,
        "dropout_rate": placeholders["dropout_rate"],
        "is_train": placeholders["is_train"],
        "enabled_node_nums": placeholders["enabled_node_nums"],
    }
    decoded_features = decode_nodes("decode_nodes", input_decoder, info)
    ### decoder for links
    decoded_adjs_list = []
    for c in range(adj_channel_num):
        decoded_adj = decode_links("decode_links_" + str(c), input_decoder, info)
        decoded_adjs_list.append(decoded_adj)
    decoded_adjs = tf.stack(decoded_adjs_list)
    decoded_adjs = tf.transpose(decoded_adjs, [1, 0, 2, 3])

    ## computing cost
    pair_adjs_sp = placeholders["adjs"][1]
    pair_features = placeholders["features"][1]
    ### array of sparse matrices to dense
    pair_adjs_list = []
    for b in range(batch_size):
        adj_y = [
            tf.sparse_tensor_to_dense(pair_adjs_sp[b][c], validate_indices=False)
            for c in range(adj_channel_num)
        ]
        pair_adjs_list.append(tf.stack(adj_y))
    pair_adjs = tf.stack(pair_adjs_list)
    #
    # adjs: batch_size x channel x N x N
    cost_features = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            labels=pair_features, logits=decoded_features
        ),
        axis=2,
    )
    # cost_features=tf.reduce_mean((pair_features-decoded_features)**2,axis=2)
    cost_links = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=pair_adjs, logits=decoded_adjs
    )
    cost_links = tf.reduce_mean(cost_links, axis=3)
    cost_links = tf.reduce_mean(cost_links, axis=2)
    # sum all costs
    cost = mask * (
        tf.reduce_mean(cost_features, axis=1) + tf.reduce_mean(cost_links, axis=1)
    )
    # cost=mask*(tf.reduce_mean(cost_features,axis=1))
    # cost=mask*(tf.reduce_sum(cost_links,axis=1))
    output = decoded_features

    cost_opt = tf.reduce_mean(cost) + klqp_loss

    cost_sum = tf.reduce_mean(cost)

    ## TODO: computing metrics
    # correct_count=mask0*tf.reshape(tf.cast(tf.greater(output1,output0),tf.float32),(-1,))
    # metrics["correct_count"]=tf.reduce_sum(correct_count)
    correct_exist = tf.cast(
        tf.equal(
            tf.reduce_max(decoded_adjs, 1) > 0.0, tf.reduce_max(pair_adjs, 1) > 0.5
        ),
        tf.float32,
    )
    # correct_count=mask*tf.reduce_sum(tf.reduce_sum(correct_exist,2),1)
    correct_count = mask * tf.reduce_mean(correct_exist, axis=[1, 2])
    metrics = {}
    metrics["correct_count"] = tf.reduce_sum(correct_count)
    ## TODO: prediction (final result)
    prediction = {
        "feature": tf.sigmoid(decoded_features),
        "dense_adj": tf.sigmoid(decoded_adjs),
    }
    return output, prediction, cost_opt, cost_sum, metrics

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_val, prediction_val, cost_val, cost_sum_val, metrics_val = sess.run(
        [output, prediction, cost_opt, cost_sum, metrics],
        feed_dict={...}  # Добавьте здесь данные для placeholders
    )
    print("Output:", output_val)
    print("Prediction:", prediction_val)
    print("Cost:", cost_val)
    print("Cost Sum:", cost_sum_val)
    print("Metrics:", metrics_val)


NameError: name 'output' is not defined